In [1]:
import pandas as pd
import numpy as np

import warnings
warnings.filterwarnings('ignore')
import requests as r

from tinkoff.invest import Client, InstrumentStatus, CandleInterval
import datetime

from bs4 import BeautifulSoup
from io import StringIO

import wget

### Параметры

In [2]:
# Токен доступа к Tinkoff API
t_token = "t.EWLRzGveOoilDzJj4yFDlRG7_3tFp1nkdoYfdauoFEW8S0lCpG_1NZgrGOYZ8AZbgmOws_JUnCAsEePzKBFRFQ"

# Определение периода выборки
start_dt = "2020-01-01"
end_dt = "2024-01-01"

# Список облигаций - определится ниже
bonds_list = []

# Список акций
tickers_list = ["GAZP", "LKOH", "SBER", "RTKM", "NVTK",
                "PLZL", "ROSN", "AFKS", "NLMK", "AGRO"]



# Список прочих инструментов
extra_list = ['IMOEX', "RTSI", "USD000000TOD", "EUR_RUB__TOD", "BRENT"]

### Пояснения к данным

0. Работаем в github репозитории, права всем нуждающимся раздам. Для начала, клоним репу - все файлы будут там. Затем добавляем свои/правим существующие в своей ветке, как готово - мержим в мастер.

1. Список облигаций выбран на портале cbonds по [ссылке](https://cbonds.ru/bonds/?emitent_country_id=0-2&emitent_type_id=0-8&status_id=5-1z141z4&kind_id=0-2&currency_id=0-2&floating_rate=0-1&order=document&dir=asc&) по фильтрам исходя из постановки задания, а именно:
- эмитент: государство;
- облигации с полностью известными размерами выплат;
- фиксированная процентная ставка;
- без оферт;
- со сроком погашения после 2024-01-01.

  Так как для доступа к cbonds нужна подписка, а делиться своей рабочей тут или запариваться с пробросом кукис ради одного гет-запросы времени нет, подходящие по критериям облигации выгружены в xlsx и положены на гит руками, чтобы можно было импортить его. В тинькофф апи не нашел поля с датой аферты - наверное, плохо искал :)
  Облигации берем равномерно по сроку погашения от максимальной к минимальной. Конечно, можно было бы выбрать их более простым способом, однако ОФЗ не так сильно отличаются друг от друга, как обыкновенные акции. 

2. Акции выбраны значительно проще, опираясь на список инструментов, определяющих индекс Мосбиржи - так как именно эти компании имеют прямой вклад в рыночную ситуацию. Постарался взять эмитентов из +- разных отраслей экономики.

3. Рыночные данные грузятся с Tinkoff API. Для воспроизведения результата нужно либо вставить свой токен и "дернуть" ячейку соответсвующую, либо грузануть уже готовые файлики с яндекс диска. Если токен пустой, файлы подгрузятся с диска

In [3]:
# Для работы с тинькофф апи нужно получить айдишник - figi

with Client(t_token) as client:
    # для бондов сохраним еще и номинал, потом уберем
    bonds_dict = client.instruments.bonds(instrument_status=InstrumentStatus.INSTRUMENT_STATUS_ALL)
    figi_dict = client.instruments.shares(instrument_status=InstrumentStatus.INSTRUMENT_STATUS_ALL)
    curr_dict = client.instruments.currencies(instrument_status=InstrumentStatus.INSTRUMENT_STATUS_ALL)
    
bonds_dict = dict([[i.ticker, [i.figi, i.nominal.units + i.nominal.nano/1e9]] for i in bonds_dict.instruments])
figi_dict = dict([[i.ticker, i.figi] for i in figi_dict.instruments])
curr_dict = dict([[i.ticker, i.figi] for i in curr_dict.instruments])

figi_dict.update(curr_dict)
# индексные figi
figi_dict.update({'RTSI': 'BBG000NJ9048', 'BRENT': 'BBG000PGXPS4'})

In [5]:
# определение списка облигаций

bonds = pd.read_excel("bonds.xlsx", engine='openpyxl')
bonds['tmp'] = bonds['Бумага'].apply(lambda x: x.split(" ")[-1][:-1])
bonds['figi'] = bonds['tmp'].apply(lambda x: bonds_dict[x][0] if x in bonds_dict.keys() else pd.NA)\
.astype("string")
bonds['nominal'] = bonds['tmp'].apply(lambda x: bonds_dict[x][1] if x in bonds_dict.keys() else pd.NA)\
# убираем номинал из словаря бондов
bond_dict = dict(zip(figi_dict.keys(), [i[0] for i in figi_dict.values()]))
bonds = (bonds[(bonds['Начало размещения'] <= '2020-01-01') & 
               (bonds.ISIN.notna()) & (bonds.figi.notna()) & (~bonds.figi.str.startswith("TCS"))]
         .reset_index(drop=True)
         .sort_values(by='Погашение', ascending=False))
bonds = bonds.loc[bonds.index.values[::bonds.shape[0] // 5],
              ['figi', 'nominal', 'ISIN', 'Бумага', 'Купон',
              'Погашение', 'Начало размещения']].head(5).reset_index(drop=True)
               
bonds_list = bonds.figi.values
bonds

,figi,nominal,ISIN,Бумага,Купон,Погашение,Начало размещения
0,BBG0000776S2,1000.0,RU000A0GN9A7,"Россия, 46020 (ОФЗ-АД, SU46020RMFS2)","1 купон - 6,95 % годовых, 2-60 купоны - 6,9% г...",2036-02-06,2006-02-15
1,BBG00B9PJ7V0,1000.0,RU000A0JVW48,"Россия, 26218 (ОФЗ-ПД, SU26218RMFS6)",1-32 купоны - 8.5% годовых,2031-09-17,2015-10-28
2,BBG00K53FBX6,1000.0,RU000A0ZYUA9,"Россия, 26224 (ОФЗ-ПД, SU26224RMFS4)",1-23 купоны - 6.9% годовых,2029-05-23,2018-02-21
3,BBG00R0Z4YW8,1000.0,RU000A1014N4,"Россия, 26232 (ОФЗ-ПД, SU26232RMFS7)",1-16 купоны - 6% годовых,2027-10-06,2019-12-04
4,BBG00D6Q7LY6,1000.0,RU000A0JWM07,"Россия, 26219 (ОФЗ-ПД, SU26219RMFS4)",1-21 купоны - 7.75% годовых,2026-09-16,2016-06-29


In [6]:
# собираем информацию по купонам облигаций
bonds_coupons = []
with Client(t_token) as client:
    for bond in bonds_list:
        coupons = client.instruments.get_bond_coupons(figi=bond,
                            from_=datetime.datetime(*[int(i) for i in start_dt.split("-")]),
                            to=datetime.datetime(*[int(i) for i in end_dt.split("-")]))
        bonds_coupons.append([[
            i.figi,
            i.coupon_date.date(),
            i.pay_one_bond.units + i.pay_one_bond.nano/1e9,
            
        ] for i in coupons.events])

bonds_coupons = pd.concat([pd.DataFrame(i, columns=['figi', 'dt', 'coupon']) for i in bonds_coupons])
bonds_coupons.figi = bonds_coupons.figi.astype("string")


bonds_coupons.head(2)

,figi,dt,coupon
0,BBG0000776S2,2020-02-12,34.41
1,BBG0000776S2,2020-08-12,34.41


In [7]:
# получение рыночных данных об облигациях
bonds_prices = []
with Client(t_token) as client:
    for bond in bonds_list:
        for year in range(int(start_dt.split("-")[0]), int(end_dt.split("-")[0])):
            prices = client.market_data.get_candles(figi=bond,
                                from_=datetime.datetime(year, 1, 1),
                                to=datetime.datetime(year + 1, 1, 1),
                                interval=CandleInterval.CANDLE_INTERVAL_DAY)
            bonds_prices.append([[
                bond,
                i.time.date(),
                i.close.units + i.close.nano/1e9] for i in prices.candles])
            
bonds_prices = pd.concat([pd.DataFrame(i, columns=['figi', 'dt', 'close']) for i in bonds_prices])
bonds_prices.figi = bonds_prices.figi.astype("string")

bonds_prices = bonds_prices.merge(right=bonds.loc[:, ['figi', 'nominal']], on='figi')


bonds_prices.head(2)

,figi,dt,close,nominal
0,BBG0000776S2,2020-01-03,102.906,1000.0
1,BBG0000776S2,2020-01-06,102.934,1000.0


In [8]:
# итоговый датафрейм по бондам с дневной дискретностью
bonds_raw = bonds_prices.merge(right=bonds_coupons, on=['figi', 'dt'], how='left')
# перевод из процентов в рубли
bonds_raw.close = bonds_raw.close / 100 * bonds_raw.nominal
bonds_raw.drop(['nominal'], axis=1, inplace=True)

bonds_raw = bonds_raw.pivot_table(columns='figi', index='dt', values=['close', 'coupon'])
bonds_raw.columns = ['_'.join(i) for i in bonds_raw.columns]
bonds_raw.to_excel("bonds_raw.xlsx", index=True)
bonds_raw.head(2)

,close_BBG0000776S2,close_BBG00B9PJ7V0,close_BBG00D6Q7LY6,close_BBG00K53FBX6,close_BBG00R0Z4YW8,coupon_BBG0000776S2,coupon_BBG00B9PJ7V0,coupon_BBG00D6Q7LY6,coupon_BBG00K53FBX6,coupon_BBG00R0Z4YW8
dt,,,,,,,,,,
2020-01-03,1029.06,1184.9,1090.99,1053.11,992.5,NaN,NaN,NaN,NaN,NaN
2020-01-06,1029.34,1183.5,1090.0,1053.5,993.0,NaN,NaN,NaN,NaN,NaN


In [9]:
# получение рыночных данных об акциях
tickers_prices = []
with Client(t_token) as client:
    for ticker in tickers_list+extra_list:
        for year in range(int(start_dt.split("-")[0]), int(end_dt.split("-")[0])):
            prices = client.market_data.get_candles(figi=figi_dict[ticker],
                                from_=datetime.datetime(year, 1, 1),
                                to=datetime.datetime(year + 1, 1, 1),
                                interval=CandleInterval.CANDLE_INTERVAL_DAY)
            tickers_prices.append([[
                ticker,
                i.time.date(),
                i.close.units + i.close.nano/1e9] for i in prices.candles])
            
tickers_prices = pd.concat([pd.DataFrame(i, columns=['ticker', 'dt', 'close']) for i in tickers_prices])
tickers_prices.ticker = tickers_prices.ticker.astype("string")
# индекс мосбиржи не нашелся в апи, подгрузим ручками по выгрузке с cbonda - положил ее в гит

imoex = pd.read_excel("imoex.xlsx")
imoex.columns = ['dt', 'close']
imoex['ticker'] = 'IMOEX'
imoex = imoex[(imoex.dt >= "2020-01-01") & (imoex.dt <= "2024-01-01")].loc[:, ['ticker', 'dt', 'close']]
imoex.dt = imoex.dt.apply(lambda x: x.date())
tickers_prices = pd.concat([tickers_prices, imoex])

tickers_prices = tickers_prices.pivot_table(columns='ticker', index='dt', values='close')
tickers_prices.columns = ["close_"+i for i in tickers_prices.columns]


imoex.head()
tickers_prices.to_excel("tickers_raw.xlsx", index=True)
tickers_prices.head(2)


,close_AFKS,close_AGRO,close_BRENT,close_EUR_RUB__TOD,close_GAZP,close_IMOEX,close_LKOH,close_NLMK,close_NVTK,close_PLZL,close_ROSN,close_RTKM,close_RTSI,close_SBER,close_USD000000TOD
dt,,,,,,,,,,,,,,,
2020-01-02,NaN,NaN,67.44,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2020-01-03,15.134,648.4,69.77,NaN,259.0,5236.14,6294.0,144.5,1264.0,7286.0,456.9,78.85,1564.18,255.0,NaN


In [ ]:
# ищем процентные ставки - за даты сорри, не хочется strptime(strftime) городить

html = r.get("https://www.cbr.ru/hd_base/zcyc_params/?UniDbQuery.Posted=True&UniDbQuery." + 
             f"From={'.'.join(start_dt.split('-')[::-1])}" +
             f"&UniDbQuery.To={'.'.join(end_dt.split('-')[::-1])}").content

perc = pd.read_html(html)[0]
perc.columns = ['dt'] + [f"%_{i[1]}y" for i in perc.columns][1:]
perc.dt = perc.dt.apply(lambda x: pd.to_datetime("-".join(x.split(".")[::-1])).date())
perc = perc.sort_values(by='dt', ascending=True)
perc.index = perc.dt
perc.drop(['dt'], axis=1, inplace=True)
perc = perc / 100
perc.to_excel("perc_raw.xlsx", index=True)


perc.head(2)

In [ ]:
# собираем итоговый датафрейм
data = (bonds_raw
        .merge(right=tickers_prices, on='dt', how='outer')
        .merge(right=perc, on='dt', how='outer')
        .sort_index()
       )
data.to_excel("dataset.xlsx")
data.tail(4).T

### Описание полей
- close_{} - цена закрытия инструмента
- coupon_{} - купон по облигациям за дату
- %_{}y - Кривая бескупонной доходности государственных облигаций на соответсвующий срок

- для акций в close суффикс - его тикер на мосбирже 

In [ ]:
# кладем все в гит

!git add .
!git commit -m "add files"
!git push